# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [32]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from datetime import date

today = date.today()

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [33]:
df1 = pd.read_csv(os.path.join('..','WeatherPy','output_data','raw_weather_data.csv'))
df = df1[['name','coord.lon','coord.lat','main.temp_max','clouds.all','main.humidity','wind.speed','sys.country','dt']]
df.head()

,name,coord.lon,coord.lat,main.temp_max,clouds.all,main.humidity,wind.speed,sys.country,dt
0,Hurricane,-113.2899,37.1753,274.15,1,79,2.06,US,1612698799
1,Busselton,115.3333,-33.6500,289.82,100,95,0.89,AU,1612698799
2,Vestmanna,-7.1664,62.1564,275.15,75,64,8.75,FO,1612698800
3,East Wenatchee Bench,-120.2812,47.4257,276.48,90,81,6.17,US,1612698800
4,Mazyr,29.2456,52.0495,261.26,36,87,2.03,BY,1612698800


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [38]:
gmaps.configure(api_key=g_key)

In [48]:
cities = df[['coord.lat','coord.lon']]
cities = cities.loc[(cities['coord.lat'] >= -90) & (df['coord.lat'] <= 90)]
cities.head()

,coord.lat,coord.lon
0,37.1753,-113.2899
1,-33.6500,115.3333
2,62.1564,-7.1664
3,47.4257,-120.2812
4,52.0495,29.2456


In [72]:
cities.loc[[4]]

,coord.lat,coord.lon
4,52.0495,29.2456


In [40]:
humidity = df["main.humidity"].astype(float)
humidity.head()

0    79.0
1    95.0
2    64.0
3    81.0
4    87.0
Name: main.humidity, dtype: float64

In [41]:
hum_max = max(humidity)
hum_max

100.0

In [49]:
fig1 = gmaps.figure()
heat = gmaps.heatmap_layer(cities, weights=humidity, dissipating=False, max_intensity=hum_max,point_radius=3)
fig1.add_layer(heat)
fig1

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [81]:
# Conditions:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.

filtered = df.dropna()
hotel_df = df.loc[(df['main.temp_max'] <= 80) & (df['main.temp_max'] >= 70) & (df['wind.speed'] <= 10) & (df['clouds.all'] == 0)]
hotel_df.reset_index(drop=True)
hotel_df

,name,coord.lon,coord.lat,main.temp_max,clouds.all,main.humidity,wind.speed,sys.country,dt


In [82]:
hotel_df = filtered
hotel_df

,name,coord.lon,coord.lat,main.temp_max,clouds.all,main.humidity,wind.speed,sys.country,dt
0,Hurricane,-113.2899,37.1753,274.15,1,79,2.06,US,1612698799
1,Busselton,115.3333,-33.6500,289.82,100,95,0.89,AU,1612698799
2,Vestmanna,-7.1664,62.1564,275.15,75,64,8.75,FO,1612698800
3,East Wenatchee Bench,-120.2812,47.4257,276.48,90,81,6.17,US,1612698800
4,Mazyr,29.2456,52.0495,261.26,36,87,2.03,BY,1612698800
...,...,...,...,...,...,...,...,...,...
582,Praxédis Guerrero,-106.0167,31.3667,280.15,1,22,3.60,MX,1612698942
583,Pangkalanbuun,111.6167,-2.6833,298.53,93,91,0.66,ID,1612698943
584,Mayya,130.2811,61.7333,233.15,48,67,3.00,RU,1612698943
585,Nouadhibou,-17.0347,20.9310,291.15,0,59,8.75,MR,1612698943


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [91]:
#type looking for
lodging = "Lodging"
#radius to search
r = 5000
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for x in range(len(hotel_df['coord.lat'])):
    lat = hotel_df.iloc[x]['coord.lat'] #sub_df.iloc[0]['A']
    lon = hotel_df.iloc[x]['coord.lon']
    coords = [lat,lon]
parameters = {"location": cities, "types": lodging, "radius": r, "key": g_key}
calls = []
for x in range(len(cities)):
    params["location"] = cities.loc[[x]]
    call = requests.get(base_url, parameters)
    calls.append(call)
for x in range(len(calls)):
    mkjson = calls[x].json()
hotel_col = []
for x in range(len(hotel_df)):
    try:
        hotel_col += [mkjson[x]['results'][0]['name'] ]
    except:
        hotel_col += [pd.NaT]
hotel_df['Hotel Name'] = hotel_col
hotel_df = hotel_df.dropna()
hotel_df = hotel_df.reset_index(drop=True)
hotel_df.rename(columns={'coord.lat': 'Lat','coord.lon': 'Lon'})
hotel_df

<ipython-input-91-b263e153c32b>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = hotel_col


,name,coord.lon,coord.lat,main.temp_max,clouds.all,main.humidity,wind.speed,sys.country,dt,Hotel Name


In [92]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: "None of [Index(['Lat', 'Lng'], dtype='object')] are in the [columns]"

In [ ]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations)

fig1.add_layer(marker_layer)

# Display figure
fig1